In [3]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image, ImageDraw
from nets import *
from utils import *
from loss import *
from files_helper import *
from train import batch_generator, read_data_from_batch
import parameter
import math
import cv2

ModuleNotFoundError: No module named 'nets'

In [ ]:
learning_rate = 0.0001
n_classes = 1
input_shape = (416,416)
batch_size = 4

dim_0 = input_shape[0] / 32
dim_1 = dim_0 * 2
dim_2 = dim_1 * 2
dim = (dim_0**2 + dim_1**2 + dim_2**2) * 3

In [ ]:
tf_x = tf.placeholder(tf.float32, [None, input_shape[0], input_shape[1], 1])

In [ ]:
with tf.device('/device:GPU:0'):
    with tf.variable_scope('detector'):
        detections, _ = yolo_v3(input_tensor=tf_x, n_classes=n_classes)
        #load_ops = load_weight(var_list=tf.global_variables(scope='detector'), weight_file=yolo_weight, for_training=False)

    boxes = get_boxes(detections, n_classes, input_shape)

In [ ]:
sess = tf.Session()
saver = tf.train.Saver()

In [ ]:
saver.restore(sess, save_path='checkpoint/')

In [ ]:
img = Image.open('../../DDSM/val/images/C_0032_1.LEFT_CC.png')
img_resized = img.resize(size=input_shape)

In [ ]:
detected_boxes = sess.run(boxes, feed_dict={tf_x:[np.expand_dims(np.array(img_resized, dtype=np.float32), axis=-1)]})

In [ ]:
filtered_boxes = non_max_suppression(detected_boxes, confidence_threshold=0.3, iou_threshold=0.1)

In [ ]:
filtered_boxes

In [ ]:
filtered_boxes[0][0][0]

In [ ]:
draw_boxes(filtered_boxes, img, detection_size=input_shape, greyscale=True)

In [ ]:
img